In [1]:
import numpy as np
import scipy

# Load data
# .item() is needed as np.load() returns a structured array that needs
# to be converted back to a dict. File should be in same location as code.
data = np.load("ex1_data.dict.npy", allow_pickle=True).item() 

print(data.keys())
print("Length of chest data: {}".format(len(data["chest"])))
print("12387th sample of wrist: {}".format(data["wrist"][12387]))


FileNotFoundError: [Errno 2] No such file or directory: 'ex1_data.dict.npy'

# Q1.1
What percentage of the total recorded time did the subject spend at Jungfraujoch (time betweenarrival and departure)?  You may use the length of the shortest data trace as base value for 100%.(max.  1pt, Full pts tol±0.2%, 10% Pts tol±3%)

In [3]:
for key in data:
        print("length of {} trace: {}".format(key, len(data[key])))

length of chest trace: 454038
length of head trace: 447464
length of wrist trace: 446831
length of ankle trace: 451008


In [4]:
import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(10, 6), sharex=True)

for key in data:
        ax[0].plot(data[key], label=key)
ax[0].legend(loc='lower right')
ax[1].plot(data['wrist'], label=key)

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Shortes trace: wrist, 446801 samples. Stay at Jungfraujoch: [95050, 300540] => 46.0%

# Q1.2
As it is not physically possible to turn on or off all devices at the same time, there is a misalign-ment between the data traces.  Shorten the traces to match the device that was turned on lastso that samples of all devices are synchronous by array index.  Do this as precisely as possible.How many samples did you remove from the beginning each trace?2(max.  2pt, Full pts tol±8samples, 10% Pts tol±50 samples)

In [9]:
import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(10, 6), sharex=True)

for key in data:
        ax[1].plot(data[key], label=key)
ax[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
data = np.load("ex1_data.dict.npy", allow_pickle=True).item() 

data['chest'] = data['chest'][:]
data['wrist'] = data['wrist'][56:]
data['ankle'] = data['ankle'][174:]
data['head']  = data['head'][119:]

In [11]:
import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(10, 6), sharex=True)

for key in data:
    if key == 'head':
        ax[1].plot(data[key]-1000, label=key)
    else:
        ax[1].plot(data[key], label=key)
ax[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Q1.3
What  was  the  sampling  rate  of  the  sensors?

Solution:
The sampling rate is 13.7.  Tolerance for full points: ±0.6 Hz points awarded fortolerances of up to ±4.5 Hz.
The sampling rate can be found using various tactics.  Using the constraints of the environment, we have to find out how much time the subject needed to travel from one altitude toanother.  One example is using the train schedule from the Eigergletscher station to Jungfraujoch (which is the only means to reach Jungfraujoch).  Under the assumption that the train picks up vertical speed 1 minute after scheduled departure (door locking etc.)  and the same at arrival, you get a sampling frequency of 12.9 for the journey up and 13.8 for the journey down.  Another way ist to identify the car ride from Grindelwald to Brunig Pass and lookup the driving time on Google Maps (yields a sampling rate of 13.8).  And there are more options similar to those (elevator ride duration and so on). To achieve a good result, averaging over several calculations may help.

# Part 2

In [12]:
elevation = dict()
for key in data:
    elevation[key] = [44307.69*(1-(p/10194000)**(0.190284)) for p in data[key]]

# Q 2.1
On what date did the subject conduct the trip to Jungfraujoch? 

The  measured  air  pressure  at  the  Sphinx  observatory  was  between  655.3  and  656.2  hPa.Looking at the chart of the weather station found in the linked resource, we can see that theonly  day  in  the  given  range  in  december  that  matches  this  range  is  16.12.2020.   Directlink  to  the  resource
https://www.meteoschweiz.admin.ch/home/messwerte.html?param=messwerte-luftdruck-qfe-10min&station=JUN&chart=day

# Q 2.2
Starting Location?
=> Grindelwald

# Q 2.3
For which of the following locations is it possible and reasonable that the subject passed throughwhile the sensors were active?  (max.  3pt)Note:  A short reasoning is required for each answer, random answers will be awarded 0 points.
* Basel (false)
* Bern (false)
* Grindelwald (true, starting loc)
* Interlaken (true, on the way back home)
* Kleine Scheidegg (46.58519, 7.96124) (false)
* Lauterbrunnen (false)
* Luzern (true, on the way back home)
* Wengen (false)
* Zurich  (true, on the way back home)

In [14]:
import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(10, 6), sharex=True)

for key in data:
    ax[0].plot(data[key], label=key)
    ax[1].plot(elevation[key], label=key)
ax[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Part 3

Q3.1
What is the maximum measured vertical speed of the elevator leading up to the viewing platformof sphinx observatory on Jungfraujoch?  Compute the value both for the ride up and down!

In [16]:
s = 134100
e = 134500

time_window = 42
summed_up = np.sum([elevation['chest'][s:e], elevation['head'][s:e], elevation['wrist'][s:e], elevation['ankle'][s:e]], axis=0)/4
speeds = [(summed_up[i]-summed_up[i-time_window])/(time_window/13.7) for i in range(time_window, len(summed_up))] 
print("max in averaged sensor traces:", max(speeds))  

maxspeeds = []
for key in data:
    summed_up = elevation[key][s:e]
    speeds = [(summed_up[i]-summed_up[i-time_window])/(time_window/13.7) for i in range(time_window, len(summed_up))] 
    print(max(speeds))  
    maxspeeds.append(max(speeds))
print("av. max across sensors:",np.average(maxspeeds))

max in averaged sensor traces: 6.194676648106336
6.429508353125892
6.33937761643642
6.216894161446667
6.22520567856637
av. max across sensors: 6.302746452393837


In [17]:
s = 210200#134000 210200
e = 210700#134500 210700

time_window = 42
summed_up = np.sum([elevation['chest'][s:e], elevation['head'][s:e], elevation['wrist'][s:e], elevation['ankle'][s:e]], axis=0)/4
speeds = [(summed_up[i]-summed_up[i-time_window])/(time_window/13.7) for i in range(time_window, len(summed_up))] 
print("max in averaged sensor traces:", min(speeds))  

maxspeeds = []
for key in data:
    summed_up = elevation[key][s:e]
    speeds = [(summed_up[i]-summed_up[i-time_window])/(time_window/13.7) for i in range(time_window, len(summed_up))] 
    print(min(speeds))  
    maxspeeds.append(min(speeds))
print("av. max across sensors:",np.average(maxspeeds))

max in averaged sensor traces: -6.585696940616097
-6.906558739522707
-6.8316349207278755
-6.395285619681355
-6.77243257298655
av. max across sensors: -6.726477963229621


# Q3.2

Whats the sampling rate of the accelerometer?

In [18]:
import numpy as np

# Load data
# .item() is needed as np.load() returns a structured array that needs
# to be converted back to a dict. File should be in same location as code.
imu_data = np.load("ex1_data_task3.dict.npy", allow_pickle=True).item() 

print(imu_data.keys())
print("Length of trace of X-axis data: {}".format(len(imu_data["X"])))
print("12387th sample of Y-axis: {}".format(imu_data["Y"][12387]))

dict_keys(['X', 'Y', 'Z'])
Length of trace of X-axis data: 4243321
12387th sample of Y-axis: 113319


# Q3.2

Whats the sampling rate of the accelerometer?

In [19]:
print(len(imu_data["X"])/len(data["chest"])*13.7)

128.03663503935792


# Q3.3a)

What percentage of the time did the subject spend walking?3You may use the length ofthe accelerometer data trace as base value for 100%.

To solve this task, we use the standard-deviation threshold algorithm presented in "Walk Detection and Step Counting on Unconstrained Smartphones", Brajdic and Harle. We calculate the total magnitude in acceleration and convert it to g. We then split it into windows of 0.8s length (or 102 samples) and compute the standard-deviation for each window. We use a slightly increased threshold compared to the paper as our IMUs are rigidly mounted and not in a smartphone. Therefore we classify each 0.8s window as walking if the std is bigger than 0.75g. This threshold adjustment from the suggested 0.6g can be done by inspecting the results for false positives and false negatives.
 After making sure, that we do not classify anything as walking while the subject sits in trains, gondolas and cars, we reach a result of 11.3%
 Tolerance for full points: +-0.5%, points awarded for tolerances of up to +-7%

In [21]:
SAMPLE_RATE = 128
imu_data['imu_magnitude'] = np.sqrt(np.square(imu_data["X"]/(2**18))+np.square(imu_data["Y"]/(2**18))+np.square(imu_data["Z"]/(2**18)))*9.81 # converted to m/s2
# Walk Detection and Step Counting on Unconstrained Smartphones, Brajdic and Harle
std_win = 102 # 0.8 seconds from the paper made to fit sampling rate with 102 samples
std_x = np.array([i for i in range(0, len(imu_data['imu_magnitude'])-std_win, std_win)])
stds = np.array([np.std(imu_data['imu_magnitude'][i:i+std_win]) for i in std_x])

imu_data['walking'] = stds>0.75 # now we have walking in there
imu_data['walking'] = imu_data['walking']*1

In [24]:
import scipy.signal
p_resampled = scipy.signal.resample(elevation['chest'], len(imu_data['imu_magnitude']))


import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=4,ncols=1,figsize=(10, 8), sharex=True)


ax[0].set_title("Elevation Profile")
ax[0].legend(loc='lower right')
ax[0].plot(p_resampled)


ax[1].set_title("Walk detection")
ax[1].plot(std_x, stds)
ax[1].plot(std_x,  imu_data['walking'])

key = 'chest'


ax[2].set_title("IMU Magnitude")
ax[2].plot(imu_data['imu_magnitude'])

ax[3].set_title("IMU Data")
ax[3].plot(imu_data['X'])
ax[3].plot(imu_data['Y'])
ax[3].plot(imu_data['Z'])


plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [25]:
#Manually enetered activities
sitting_intervals = []

#train and gondola rides
sitting_intervals.append([0, 335000])
sitting_intervals.append([543000, 660000])
sitting_intervals.append([710000, 794000])
sitting_intervals.append([824000, 896000])
sitting_intervals.append([2806000, 2974000])
sitting_intervals.append([3030000, 3138000])
sitting_intervals.append([3231000, 4243320])

#Remove those periods from the walking detection
for interval in sitting_intervals:
    imu_data['walking'][int(interval[0]/std_win):int(interval[1]/std_win)]=0

In [26]:
print("Percentage of walking", sum(imu_data['walking'])/len(std_x))

Percentage of walking 0.11338669743515781


# Q3.3b)

How many steps did the subject take throughout the day? 

Again  we  take  an  algorithm  from  ”Walk  Detection  and  Step  Counting  onUnconstrained Smartphones”, Brajdic and Harle as an example:  windowed peak detection. First we smoothen the magnitude signal with a running average filter of 40 samples and then run a peak detection.  For the latter it is possible to trim the parameters to visually reduce false positives and false negatives.  Of course we only consider peaks during periods of walking.  Using a minimum prominence of 0.1g and a minimum distancebetween peaks of 0.52s, we count 5596 steps. Tolerance for full points:±400 steps, points awarded for tolerances of up to±2500 steps.

In [29]:
imu_data['magnitude_smooth'] = scipy.ndimage.uniform_filter1d(imu_data['imu_magnitude'], 40) # .31s mov av As described in paper ”Walk Detection and Step Counting on Unconstrained Smartphones”, Brajdicand Harle
pks,_ = scipy.signal.find_peaks(imu_data['magnitude_smooth'], prominence=0.1, distance=67) #

for i, walk in enumerate(imu_data['walking']):
    if walk == 0:
        pks = np.delete(pks, np.logical_and(pks >= i*std_win, pks < (i+1)*std_win))
        
print("number of steps:", len(pks))        

number of steps: 5596


In [31]:
import scipy
p_resampled = scipy.signal.resample(elevation['chest'], len(imu_data['imu_magnitude']))

import matplotlib.pyplot as plt
%matplotlib widget

fig,ax=plt.subplots(nrows=4,ncols=1,figsize=(10, 8), sharex=True)

ax[0].set_title("Elevation Profile")
ax[1].set_title("Walk detection")
ax[2].set_title("IMU Magnitude / Step Count")
ax[3].set_title("IMU Data")


ax[0].legend(loc='lower right')
ax[0].plot(p_resampled)
#ax[2].plot(vt.butter_bandpass_filter(height, 0.0001, 0.5, 13.7, 1))
ax[1].plot(std_x, stds)
ax[1].plot(std_x,  imu_data['walking'])

key = 'chest'


ax[2].plot(imu_data['imu_magnitude'], label="IMU magn.")
ax[2].plot(imu_data['magnitude_smooth'], label="IMU magn. averaged")
ax[2].plot(pks, imu_data['magnitude_smooth'][pks],  'or', label="Detected steps")
ax[2].legend(loc='lower right')

ax[3].plot(imu_data['X'])
ax[3].plot(imu_data['Y'])
ax[3].plot(imu_data['Z'])


plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
